In [86]:
import time
import os
import pandas as pd
from urllib.request import urlretrieve
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
#2024년 새로 추가된 라이브러리
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

def fb_ad_crop():
    DATE = []           # 광고 게시일자
    ID = []             # 광고 아이디
    CONTENT = []        # 광고 글(내용)
    HREF = []           # 광고 랜딩페이지
    img_src = []        # 이미지 소스
    video_src = []      # 동영상 소스
    driver_path = 'chromedriver/chromedriver.exe'
    folder_path = 'C:/FB/{}'


    # 원하는 검색어 입력
    search = input("검색어를 입력해주세요:")

    # 브라우저 드라이버 path 설정
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # 브라우저 창 최대화
    chrome_service = Service(executable_path=driver_path)
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    # 오픈 브라우저 주소
    driver.get('https://www.facebook.com/ads/library/?active_status=all&ad_type=all&country=KR&q=' + search + '&sort_data[direction]=desc&sort_data[mode]=relevancy_monthly_grouped&search_type=keyword_unordered&media_type=all')
    driver.implicitly_wait(5)
    
    # 페이지 모든 요소를 보기 위해 스크롤 끝까지 내려주는 코드 (스크롤이 내려가야 아래 html이 생성되기 때문)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(2)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    fb_ad = driver.find_elements(By.CSS_SELECTOR, '._7jvw ')

    # 저장폴더 경로 설정
    folder_path = folder_path.format(search)

    # 경로 없으면 경로 생성
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)

    # 모든광고에 대해 한 광고씩 크롤링 진행
    for i in range(len(fb_ad)):
        ad_info = fb_ad[i].find_elements(By.CSS_SELECTOR, '.xeuugli')
        DATE.append(ad_info[0].text)    # 게시일자
        ID.append(ad_info[1].text)      # 게시 ID

        ad_text = fb_ad[i].find_element(By.CSS_SELECTOR, '._7jyr')
        CONTENT.append(ad_text.text)    # 광고 글

        # 이미지, 동영상 추출
        try:
            ad_images = fb_ad[i].find_elements(By.CSS_SELECTOR, 'img.x642log') #이미지 소스 찾기

            # 각 이미지의 src 속성 추출하여 리스트에 추가
            for ad_img in ad_images:
                img_src.append(ad_img.get_attribute('src'))
        except:
            pass

        try:
            ad_img = fb_ad[i].find_element(By.TAG_NAME, 'video').get_attribute('src')            # 동영상 소스 찾기
            video_src.append(ad_img)
        except:
            pass

        # 랜딩페이지에 대한 href 주소 크롭
        try:
            ad_href = fb_ad[i].find_element(By.TAG_NAME, 'a').get_attribute('href')                  # 랜딩페이지 주소 찾기
            HREF.append(ad_href)
        except:
            HREF.append("")
    
    import re

    # ID를 추출하는 정규표현식 패턴
    id_pattern = re.compile(r'ID: (\d+)')

    # 날짜를 추출하는 정규표현식 패턴
    date_pattern_single = re.compile(r'(\d{4})\. (\d{1,2})\. (\d{1,2})\.')
    date_pattern_range = re.compile(r'(\d{4})\. (\d{1,2})\. (\d{1,2})\.~(\d{4})\. (\d{1,2})\. (\d{1,2})\.')

    # ID와 날짜를 저장할 리스트
    IDs = []
    dates = []

    # DATE 리스트에서 각 문자열에서 ID와 날짜 추출
    for entry in DATE:
        # ID 추출
        id_match = id_pattern.search(entry)
        if id_match:
            IDs.append(id_match.group(1))

        # 날짜 추출
        date_match_single = date_pattern_single.search(entry)
        if date_match_single:
            year, month, day = date_match_single.groups()
            formatted_date = f"{year[2:]}{int(month):02d}{int(day):02d}"
            dates.append(formatted_date)
        else:
            date_match_range = date_pattern_range.search(entry)
            if date_match_range:
                year1, month1, day1, year2, month2, day2 = date_match_range.groups()
                formatted_date = f"{year1[2:]}{int(month1):02d}{int(day1):02d}~{year2[2:]}{int(month2):02d}{int(day2):02d}"
                dates.append(formatted_date)
            
    # 추출된 IDs 출력
 

    # 이미지 저장
    for index, link in enumerate(img_src):
        urlretrieve(link, folder_path + '/{}_{}.jpg'.format(search, index + 1))
    # 동영상 저장
    for index, link in enumerate(video_src):
        urlretrieve(link, folder_path + '/{}_{}.mp4'.format(search, index + 1))

    # 브라우저 종료
    driver.quit()

    # 데이터 csv 변환
    data = {'DATE': dates, 'ID': IDs, 'CONTENT': CONTENT, 'HREF': HREF}
    data = pd.DataFrame(data)

    data.to_csv(folder_path + '/{}.csv'.format(search), index=False, encoding="utf-8-sig")


In [87]:
fb_ad_crop()

검색어를 입력해주세요:풀무원


In [85]:
data

,DATE,ID,CONTENT,HREF
0,240709,305130369259791,안 써본 사람은 있어도\n한 번만 써본 사람은 없다는 품절대란템🛒\n\n화제의 중심...,https://facebook.com/HDDFS.official
1,240709,1637642647068647,안 써본 사람은 있어도\n한 번만 써본 사람은 없다는 품절대란템🛒\n\n화제의 중심...,https://facebook.com/HDDFS.official
2,240522,474996411757365,🏷️린넨 브이티 (4컬러)\n이번주 금요일까지 10% 할인 진행해요😘\n여유핏이구요...,https://l.facebook.com/l.php?u=https%3A%2F%2Fw...
3,240427,416094711359357,🔥 🤭😬아름다운 소녀가 청소부로 일한 첫날 상사로부터 위협을 받았습니다.\n하지만 ...,https://facebook.com/100090532125047


In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from urllib.request import urlretrieve
import pandas as pd
import re

# 필요한 리스트 초기화
DATE = []           # 광고 게시일자
ID = []             # 광고 아이디
CONTENT = []        # 광고 글(내용)
HREF = []           # 광고 랜딩페이지
img_src = []        # 이미지 소스
video_src = []      # 동영상 소스

# 드라이버 및 폴더 경로 설정
driver_path = 'chromedriver/chromedriver.exe'
folder_path = 'C:/FB/{}'

# 원하는 검색어 입력
search = input("검색어를 입력해주세요:")

# 브라우저 드라이버 path 설정
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # 브라우저 창 최대화
chrome_service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# 오픈 브라우저 주소
driver.get('https://www.facebook.com/ads/library/?active_status=all&ad_type=all&country=KR&q=' + search + '&sort_data[direction]=desc&sort_data[mode]=relevancy_monthly_grouped&search_type=keyword_unordered&media_type=all')
driver.implicitly_wait(5)

# 페이지 모든 요소를 보기 위해 스크롤 끝까지 내려주는 코드 (스크롤이 내려가야 아래 html이 생성되기 때문)
scroll_pause_time = 2
max_scrolls = 10  # 최대 스크롤 횟수 설정
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(max_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 광고 요소 찾기
fb_ad = driver.find_elements(By.CSS_SELECTOR, '._7jvw')

# 저장폴더 경로 설정
folder_path = folder_path.format(search)

# 경로 없으면 경로 생성
if not os.path.isdir(folder_path):
    os.makedirs(folder_path)

# 모든 광고에 대해 한 광고씩 크롤링 진행
for i in range(len(fb_ad)):
    ad_info = fb_ad[i].find_elements(By.CSS_SELECTOR, '.xeuugli')
    DATE.append(ad_info[0].text)    # 게시일자
    ID.append(ad_info[1].text)      # 게시 ID
    
    ad_text = fb_ad[i].find_element(By.CSS_SELECTOR, '._7jyr')
    CONTENT.append(ad_text.text)    # 광고 글

    # 이미지, 동영상 추출
    try:
        ad_images = fb_ad[i].find_elements(By.CSS_SELECTOR, 'img.x642log')  # 이미지 소스 찾기
        for ad_img in ad_images:
            img_src.append(ad_img.get_attribute('src'))
    except:
        pass

    try:
        ad_video = fb_ad[i].find_element(By.TAG_NAME, 'video').get_attribute('src')  # 동영상 소스 찾기
        video_src.append(ad_video)
    except:
        pass

    # 랜딩페이지에 대한 href 주소 크롭
    try:
        ad_href = fb_ad[i].find_element(By.TAG_NAME, 'a').get_attribute('href')  # 랜딩페이지 주소 찾기
        HREF.append(ad_href)
    except:
        HREF.append("")

# ID와 날짜를 추출하는 정규표현식 패턴
id_pattern = re.compile(r'ID: (\d+)')
date_pattern_single = re.compile(r'(\d{4})\. (\d{1,2})\. (\d{1,2})\.')
date_pattern_range = re.compile(r'(\d{4})\. (\d{1,2})\. (\d{1,2})\.~(\d{4})\. (\d{1,2})\. (\d{1,2})\.')

# ID와 날짜를 저장할 리스트
IDs = []
dates = []

# DATE 리스트에서 각 문자열에서 ID와 날짜 추출
for entry in DATE:
    # ID 추출
    id_match = id_pattern.search(entry)
    if id_match:
        IDs.append(id_match.group(1))

    # 날짜 추출
    date_match_single = date_pattern_single.search(entry)
    if date_match_single:
        year, month, day = date_match_single.groups()
        formatted_date = f"{year[2:]}{int(month):02d}{int(day):02d}"
        dates.append(formatted_date)
    else:
        date_match_range = date_pattern_range.search(entry)
        if date_match_range:
            year1, month1, day1, year2, month2, day2 = date_match_range.groups()
            formatted_date = f"{year1[2:]}{int(month1):02d}{int(day1):02d}~{year2[2:]}{int(month2):02d}{int(day2):02d}"
            dates.append(formatted_date)

# 이미지 저장
for index, link in enumerate(img_src):
    urlretrieve(link, folder_path + '/{}_{}.jpg'.format(search, index + 1))

# 동영상 저장
for index, link in enumerate(video_src):
    urlretrieve(link, folder_path + '/{}_{}.mp4'.format(search, index + 1))

# 브라우저 종료
driver.quit()

# 데이터 csv 변환
data = {'DATE': dates, 'ID': IDs, 'CONTENT': CONTENT, 'HREF': HREF}
data = pd.DataFrame(data)

data.to_csv(folder_path + '/{}.csv'.format(search), index=False, encoding="utf-8-sig")


검색어를 입력해주세요:삼성
